# Arbitrary Style Transfer
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mdehling/ghiasi-arbitrary-style-transfer/blob/main/arbitrary-style-transfer.ipynb)

This notebook lets you try out image stylization using the approach described
by Ghiasi, Lee, Kudlur, Dumoulin, and Shlens in their article _Exploring the
Structure of a Real-Time, Arbitrary Neural Artistic Stylization Network_.  For
more details, see my github repositories [`ghiasi-arbitrary-style-transfer`](
https://github.com/mdehling/ghiasi-arbitrary-style-transfer) and
[`nstesia`](https://github.com/mdehling/nstesia).

In [ ]:
# At the time of writing, the versions of pip and setuptools provided by colab
# do not have full `pyproject.toml` support --- they must be updated before
# installing the nstesia package.  This cell will do just that.
try:
    from google import colab

    # Pull everything else from the repository in to the Colab environment.
    !git config --global init.defaultBranch main
    !git init .
    !git remote add origin https://github.com/mdehling/ghiasi-arbitrary-style-transfer.git
    !git pull --depth=1 origin main

    # These are just to avoid some scary-looking (but harmless) error messages.
    !pip uninstall -q -y numba
    !pip install -q 'jedi>=0.10'

    # Minimum versions required for PEP-660 support.
    !pip install -q 'pip>=21.3' 'setuptools>=64'

    # Finally install the one we came for.
    !pip install -q -r requirements-colab.txt

except ImportError:
    pass

In [ ]:
from IPython.display import display

from os import environ as env
env['TF_CPP_MIN_LOG_LEVEL'] = '2'               # hide info & warnings
env['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'       # grow GPU memory as needed

import tensorflow as tf
import nstesia as nst

## Loading the Model

In [ ]:
# This will download and unpack the saved model.
![ ! -e saved ] && \
    gdown 1rZrUQLriaJhsYFFUojotm5LRiCNikow5 && \
    tar xjf saved.tar.bz2 && \
    rm -f saved.tar.bz2

In [ ]:
model = nst.ghiasi_2017.StyleTransferModel.from_saved('saved/model-pbn-48')

## Simple Image Stylization

In [ ]:
def show_image(image_tensor):
    display(tf.keras.utils.array_to_img(tf.squeeze(image_tensor,axis=0)))

In [ ]:
content_image = nst.io.load_image('img/content/chicago.jpg')
style_image = nst.io.load_image('img/style/van-gogh-self-portrait.jpg')
pastiche_image = model( (content_image, style_image) )
show_image(pastiche_image)

## Content-Style Grids

In [ ]:
def content_style_grid(content_images, style_images):
    """
    Show a grid of stylizations with content images as rows, styles as columns.

    Args:
        content_images:
            A list of image tensors representing the content images.
        style_images:
            A list of image tensors representing the style images.

    Returns:
        An image representing the grid of stylizations.
    """
    images = [None] + style_images
    for content_image in content_images:
        images += [content_image] + [
            model((content_image,style_image)) for style_image in style_images
        ]

    return nst.image.grid(images, ncols=len(style_images)+1)

In [ ]:
# img/results/content-style-matrix-1.png
content_images = [
    nst.io.load_image('img/content/sunflower.jpg'),
    nst.io.load_image('img/content/bochum.jpg'),
]
style_images = [
    nst.io.load_image('img/style/wescoat-candy.jpg'),
    nst.io.load_image('img/style/picasso-briqueterie-a-tortosa.jpg'),
    nst.io.load_image('img/style/picabia-udnie.jpg'),
]
grid_image = content_style_grid(content_images, style_images)
show_image(grid_image)

In [ ]:
# img/results/content-style-matrix-2.png
content_images = [
    nst.io.load_image('img/content/brad.jpg'),
    nst.io.load_image('img/content/karya.jpg'),
]
style_images = [
    nst.io.load_image('img/style/petitjean-femmes-au-bain.jpg'),
    nst.io.load_image('img/style/delaunay-portrait-de-metzinger.jpg'),
    nst.io.load_image('img/style/munch-skrik.jpg'),
]
grid_image = content_style_grid(content_images, style_images)
show_image(grid_image)